# **Trabajo de Fin de Grado**
## **Predicción del Precio del Oro mediante Modelos de Aprendizaje Automático**

**Autor:** Sebastián Malbaceda Leyva
**Tutor:** Alberto Rubio
**Grado en Ingeniería Informática – Universitat Autònoma de Barcelona**
**Campus Bellaterra, Septiembre 2025**

---

## **Descripción General del Proyecto**

El presente **Trabajo de Fin de Grado (TFG)** tiene como propósito el diseño e implementación de un **sistema predictivo del precio del oro a corto plazo**, empleando técnicas de **Machine Learning (ML)** y **análisis de series temporales**.

El proyecto se enmarca en la línea de *modelización financiera* y análisis cuantitativo de activos, buscando **comparar la eficacia de distintos modelos predictivos** —lineales, basados en árboles y estadísticos clásicos— para determinar su viabilidad, precisión y robustez en el contexto del mercado del oro.

Históricamente, el oro ha sido considerado un **activo refugio**, especialmente en períodos de inestabilidad económica o política. Sin embargo, su valor está condicionado por una gran cantidad de factores interrelacionados, como la **política monetaria**, las **expectativas de inflación**, la **fortaleza del dólar estadounidense**, el **comportamiento de los mercados bursátiles** y los **indicadores de riesgo global**.

Predecir su precio representa, por tanto, un reto de gran complejidad analítica:
- Es un activo **altamente volátil**, afectado por variables de distinta naturaleza y frecuencia temporal.
- Combina **movimientos especulativos de corto plazo** con **factores macroeconómicos estructurales**.
- Requiere integrar datos **heterogéneos y multiescalares** (diarios, mensuales, anuales).

Este TFG busca abordar esa complejidad construyendo un pipeline reproducible de *Machine Learning* que permita **capturar relaciones causales y patrones dinámicos**, generando una herramienta predictiva aplicable a la toma de decisiones en el ámbito financiero.

---

## **Estructura del Proyecto**

El desarrollo del proyecto se organiza en siete etapas principales:

1. **Definición del problema y recopilación de datos.**
2. **Preprocesamiento y armonización temporal de las series.**
3. **Análisis exploratorio (EDA) y selección de características.**
4. **Ingeniería de características (Feature Engineering).**
5. **Entrenamiento de modelos predictivos.**
6. **Evaluación y validación temporal.**
7. **Interpretabilidad de resultados y conclusiones.**

---

# **1) Definición del Problema y Datos**

---

## 🎯 **Contexto del Problema**

El objetivo principal de este trabajo es **predecir el precio spot del oro (XAU/USD)** utilizando técnicas de *Machine Learning* y *series temporales multivariadas*.
El sistema propuesto debe ser capaz de **estimar movimientos futuros del oro a corto plazo**, integrando información de los mercados financieros, variables macroeconómicas y factores de sentimiento global.

El precio del oro se ve afectado por una amplia gama de determinantes:

- **Política monetaria y tasas de interés** (FED Funds Rate, rendimientos de bonos).
- **Índice del dólar (DXY)** y otros tipos de cambio.
- **Indicadores bursátiles** (S&P 500, VIX, índices de commodities).
- **Variables macroeconómicas** (PIB, inflación, desempleo, oferta monetaria).
- **Flujos de inversión** en ETFs y posiciones en futuros (CFTC, SPDR Gold Shares).
- **Riesgo e incertidumbre geopolítica** (EPU, GPR).
- **Factores sociales y de comportamiento** (Google Trends, confianza del consumidor).

Por la naturaleza del activo, el proyecto se clasifica como un **problema de regresión de series temporales**, donde el objetivo es predecir un valor numérico continuo en función de un conjunto amplio de variables históricas y exógenas.

---

## 📂 **Datos Utilizados**

Durante la fase inicial se recolectaron **más de 60 variables válidas**, verificadas y documentadas, provenientes de fuentes oficiales y plataformas financieras reconocidas.
Estas variables representan una visión completa del entorno económico, financiero y de mercado del oro.

### 🔸 Principales categorías y ejemplos de variables

| Categoría | Ejemplos de Variables | Fuente | Frecuencia |
|------------|----------------------|---------|-------------|
| **Precio del Oro (Target)** | Oro spot (XAU/USD), Oro futuros (GCZ5) | Investing.com | Diaria |
| **Metales Preciosos** | Plata (XAG/USD), Platino (XPT/USD), Paladio (XPD/USD), Cobre (PCOPPUSDM) | FRED / Investing | Diaria / Mensual |
| **Índices Financieros** | S&P 500, DXY, VIX, CRB, Bloomberg Commodity Index | FRED / Investing | Diaria |
| **Energía y Materias Primas** | WTI, Brent (spot y futuros) | FRED / Investing | Diaria |
| **Macroeconomía** | Inflación (CPI), PIB, M2, Producción Industrial, Desempleo, Ventas minoristas | FRED | Mensual / Trimestral |
| **Bonos y Tasas de Interés** | DGS2, DGS10, T10YIE, DFII10 (TIPS), Credit Spread | FRED | Diaria |
| **Riesgo Global** | EPU, GPR, Financial Stress Index | policyuncertainty.com / Matteo Iacoviello / FRED | Diaria / Semanal |
| **Sentimiento del Consumidor** | UMCSENT, CCI, Google Trends “Gold” | FRED / OECD / Google Trends | Mensual |
| **Flujos e Inversión** | ETF holdings (GLD), posiciones CFTC, balance de la FED | SPDR / CFTC / FRED | Diaria / Semanal |
| **Divisas y Criptoactivos** | USD/CNY, EUR/USD, USD/JPY, Bitcoin | Investing / FRED | Diaria |

El conjunto final se organiza en tres niveles de frecuencia:
- **≈40 variables diarias**
- **≈13 variables mensuales**
- **1 variable anual (déficit fiscal)**

---

## 🧮 **Preprocesamiento de los Datos**

Dada la naturaleza heterogénea de las fuentes, fue necesario aplicar un proceso de **armonización temporal, imputación y normalización** antes del análisis exploratorio y modelado.

### 🔹 Pasos principales

1. **Conversión de frecuencias:**
   Todas las series se alinearon en una misma escala temporal (diaria), manteniendo la coherencia temporal mediante:
   - *Forward-fill* para series diarias.
   - *Interpolación lineal* o spline para series mensuales, trimestrales y anuales.

2. **Tratamiento de valores faltantes:**
   Aplicación de métodos de imputación dependientes de la naturaleza de cada serie, y creación de indicadores binarios en casos relevantes (*missingness informative*).

3. **Normalización y escalado:**
   Uso de `StandardScaler` o `RobustScaler` para asegurar comparabilidad entre magnitudes.

4. **Control de colinealidad:**
   Evaluación mediante correlaciones de Pearson/Spearman y **Variance Inflation Factor (VIF)**, eliminando redundancias o combinando variables.

5. **Homogeneización de unidades y formatos:**
   Conversión de todas las magnitudes a dólares estadounidenses (USD) y ajuste de porcentajes, rendimientos y tasas.

6. **Consolidación final:**
   Obtención de un **dataset multivariado limpio, sincronizado y sin valores nulos**, preparado para el EDA y el modelado.

---

## 🎯 **Objetivo del Modelo**

El objetivo del modelo es **predecir el precio spot del oro a corto plazo** (1–5 días), integrando variables históricas del propio oro y determinantes externos.
El enfoque busca capturar tanto relaciones lineales como no lineales y medir la influencia relativa de cada grupo de factores.

### Tipo de problema
- **Regresión supervisada multivariante de series temporales.**

### Variable objetivo (Target)
- `gold_spot` — precio spot del oro (USD/oz).

### Métricas de evaluación
- **RMSE (Root Mean Squared Error)**: penaliza errores grandes.
- **MAE (Mean Absolute Error)**: magnitud media del error en unidades monetarias.
- **R² (Coeficiente de determinación)**: porcentaje de varianza explicada.

### Validación
- **Cross-validation temporal (TimeSeriesSplit)** para evaluar el rendimiento a lo largo del tiempo, evitando *data leakage*.

---

## ⚙️ **Metodología General y Guía de Trabajo**

El pipeline metodológico adoptado sigue una estructura coherente con las mejores prácticas de la ciencia de datos moderna (según la *Guía Definitiva ML*):

1. **Obtención de datos** — consolidación de fuentes externas (FRED, Investing, WGC, CFTC, SPDR).
2. **Preprocesamiento** — imputación, limpieza y homogeneización temporal.
3. **Selección de características** — combinación de métodos univariados y multivariados:
   - Correlaciones (Pearson/Spearman).
   - Información mutua y F-test.
   - Importancia de árboles (Random Forest, XGBoost).
   - Reducción de dimensionalidad con PCA.
   - Cálculo de VIF para multicolinealidad.
4. **Feature Engineering temporal** — creación de variables derivadas:
   - Lags, medias móviles, volatilidad, retornos logarítmicos.
   - Variables estacionales (día, mes).
   - Indicadores técnicos derivados (RSI, curva 10Y−2Y).
5. **Modelado** — comparación de modelos:
   - *Lineales:* Regresión, Ridge, Lasso.
   - *Basados en árboles:* RandomForest, XGBoost, LightGBM, CatBoost.
   - *Series temporales:* ARIMA, SARIMA.
6. **Evaluación y validación temporal:**
   MSE, RMSE, MAE, análisis de tendencia y direccionalidad.
7. **Interpretabilidad:**
   SHAP values, permutation importance, análisis de sensibilidad.

---

## 🌍 **Fuentes de los Datos**

| Fuente | Descripción | Enlace |
|--------|--------------|--------|
| FRED (Federal Reserve Economic Data) | Datos macroeconómicos y financieros (PIB, inflación, tasas, producción industrial, empleo). | https://fred.stlouisfed.org |
| Investing.com | Precios spot y futuros de metales, índices, commodities y divisas. | https://www.investing.com |
| World Gold Council (WGC) | Datos estructurales sobre demanda, reservas y oferta de oro. | https://www.gold.org |
| CFTC (Commitments of Traders Reports) | Posiciones netas de traders en los mercados de futuros. | https://www.cftc.gov/MarketReports |
| SPDR Gold Shares (ETF GLD) | Flujos de ETFs respaldados por oro físico. | https://www.spdrgoldshares.com |
| Economic Policy Uncertainty Index (EPU) | Nivel de incertidumbre económica global. | https://www.policyuncertainty.com |
| Geopolitical Risk Index (GPR) | Medición del riesgo geopolítico mundial. | https://www.matteoiacoviello.com/gpr.htm |
| Google Trends | Interés de búsqueda del término “Gold”. | https://trends.google.com |

---

## 📚 **Conclusión del Punto 1**

En este primer bloque se ha establecido el **contexto, los objetivos y la base de datos del proyecto**.
El oro, como activo multifactorial, requiere un tratamiento analítico riguroso y la integración de fuentes heterogéneas de información.
El conjunto de datos recolectado y preprocesado constituirá el punto de partida para el **Análisis Exploratorio de Datos (EDA)**, donde se identificarán patrones, correlaciones y variables clave antes del entrenamiento de los modelos predictivos.

---

> 💡 **A continuación:**
> Se procederá con el **EDA (Etapa 2)**, donde se analizarán las distribuciones, correlaciones y relaciones entre variables utilizando herramientas visuales y estadísticas avanzadas.
